# Point Cloud Registration

In order to lower the uncertainty of calculations done with `py4dgeo`, point cloud registration should be performed to tightly align the point clouds. `py4dgeo` supports this in two ways:

* It allows you to apply arbitrary affine transformations to epochs and stores the transformation parameters as part of the epoch. You can use the tooling of your choice to calculate these.
* It provides a number of registration algorithms that allow you to calculate the transformation directly in `py4dgeo`. Currently, this is limited to standard ICP.

This notebook show cases both ways of usage.

Note: Be aware that an initial transformation is required that roughly aligns the two point clouds. 

In [ ]:
import py4dgeo
import numpy as np

We load two epochs and register them to each other:

In [ ]:
epoch1 = py4dgeo.read_from_xyz("plane_horizontal_t1.xyz")  # replace with own data
epoch2 = py4dgeo.read_from_xyz("plane_horizontal_t2.xyz")  # replace with own data

# Return value
Registration algorithms use a simple function call interface. It returns an affine transformation as a `3x4` matrix which contains the Rotation matrix $\mathbf{R}$ and the translation vector $\mathbf{t}$:

$$
\mathbf{T} = \left(\begin{array}{ccc}r_{11}&r_{12}&r_{13}&t_1\\r_{21}&r_{22}&r_{23}&t_2\\r_{31}&r_{32}&r_{33}&t_3\end{array}\right)
$$

When calculating and applying such transformations in `py4dgeo`, it is always possible to specify the *reduction point* $\mathbf{x_0}$. This allows shifting coordinates towards the origin before applying rotations - leading to a numerically robust operation. The algorithm for a transformation is:

$$
\mathbf{Tx} = \left(\mathbf{R}(\mathbf{x}-\mathbf{x_0})\right) + \mathbf{t} + \mathbf{x_0}
$$

In [ ]:
trafo = py4dgeo.iterative_closest_point(
    epoch1, epoch2, reduction_point=np.array([0, 0, 0])
)

This transformation can then be used to transform `epoch2`:

In [ ]:
epoch2.transform(trafo)

## or use an external transformation matrix
# external_trafo = np.loadtxt("trafo.txt") # replace with own data
# epoch2.transform(external_trafo)

The `Epoch` class records applied transformations and makes them available through the `transformation` property:

In [ ]:
epoch2.transformation

Finally, we can export the transformed epoch to inspect the registration quality e.g. in CloudCompare.

In [ ]:
# epoch2.save("plane_horizontal_t2_transformed.xyz") # replace with own data

## Available registration algorithms

Previously, only the standard point-to-point ICP was available, but a registration algorithm based on the point-to-plane algorithm has now been implemented. It is based on the article "Supervoxel-based targetless registration and identification of stable areas for deformed point clouds" by Yuhui Yang and Volker Schwieger.

In [ ]:
epoch1 = py4dgeo.read_from_las(
    "plane_horizontal_t1_w_normals.laz",
    normal_columns=["NormalX", "NormalY", "NormalZ"],
)  # replace with own data
epoch2 = py4dgeo.read_from_las(
    "plane_horizontal_t2_w_normals.laz",
    normal_columns=["NormalX", "NormalY", "NormalZ"],
)  # replace with own data

For this method normals must be calculated or added before using function! Also, for the function to work, several arguments must be specified on the input:
     
`reference_epoch`: The reference epoch to match with.

`epoch`:  The epoch to be transformed to the reference epoch.

`initial_distance_threshold`: The upper boundary of the distance threshold in the iteration. It can be (1) an empirical value manually set by the user according to the approximate accuracy of coarse registration, or (2) calculated by the mean and standard of the nearest neighbor distances of all points.

`level_of_detection`: The lower boundary (minimum) of the distance threshold in the iteration. It can be  (1) an empirical value manually set by the user according to the approximate uncertainty of laser scanning measurements in different scanning configurations and scenarios (e.g., 1 cm for TLS point clouds in short distance and 4 cm in long distance, 8 cm for ALS point clouds, etc.), or (2) calculated by estimating the standard deviation from local modeling (e.g., using the level of detection in M3C2 or M3C2-EP calculations).

`reference_supervoxel_resolution`: The approximate size of generated supervoxels for the reference epoch. It can be (1) an empirical value manually set by the user according to different surface geometries and scanning distance (e.g., 2-10 cm for indoor scenes, 1-3 m for landslide surface), or (2) calculated by 10-20 times the average point spacing (original resolution of point clouds). In both cases, the number of points in each supervoxel should be at least 10 (i.e., min_svp_num = 10).

`supervoxel_resolution`:  The same as `reference_supervoxel_resolution` but for a different epoch.

`min_svp_num`: Minimum number of points for supervoxels to be taken into account in further calculations.

This function returns the final version of transformation matrix in the same shape as [here](#return-value). This matrix might be used as it is or for futher postprocessing

In [ ]:
trafo = py4dgeo.icp_with_stable_areas(
    epoch1,
    epoch2,
    initial_distance_threshold=10,
    level_of_detection=10,
    reference_supervoxel_resolution=0.2,
    supervoxel_resolution=0.2,
    min_svp_num=1,
)

#epoch2.transform(trafo)